# Original SDAC Code

In [1]:
import numpy as np
import gurobipy as gp
import math
import sympy
import contextlib
import time
import os
from utils import result, METHODS, INF, EPS
from mps_reader_preprocessor import read_mps_preprocess
from polyhedral_model import PolyhedralModel
from polyhedron import Polyhedron

## Read in Polyhedron P

In [2]:
# problems = ['adlittle', 'afiro', 'agg2','agg3','agg', 'bandm',
#  'beaconfd', 'blend', 'bnl1', 'boeing1', 'boeing2', 'bore3d', 'brandy', 'capri',
#  'cre-c', 'cycle', 'degen2', 'degen3', 'e226', 'etamacro', 'fffff800', 'finnis',
#  'fit1d', 'fit1p', 'forplan', 'ganges', 'gfrd-pnc', 'grow15', 'grow22', 'grow7',
#  'israel', 'kb2', 'ken-07', 'lotfi', 'maros', 'modszk1', 'pds-02', 'perold',
#  'pilot4', 'pilotnov', 'qap08', 'recipe', 'sc105', 'sc205', 'sc50a',
#  'sc50b', 'scagr25', 'scagr7', 'scfxm1', 'scfxm2', 'scfxm3', 'scorpion', 'scrs8',
#  'scsd1', 'scsd6', 'scsd8', 'sctap1', 'sctap2', 'sctap3', 'seba', 'share1b',
#  'share2b', 'shell', 'ship04l', 'ship04s', 'ship08l', 'ship08s', 'ship12l', 'ship12s',
#  'sierra', 'stair', 'standata', 'standgub', 'standmps', 'stocfor1', 'stocfor2', 'tuff',
#  'vtp_base', 'wood1p']


# # mps_fn='C:\Users\DillW\OneDrive - The University of Colorado Denver\Documents\GitHub\simplex_like_alg\netlib_lp_subset\kb2'
# for i in problems:
#     problem_dir = 'netlib_lp_subset'
#     # problem = 'afiro'
#     mps_fn=os.path.join(problem_dir, i)
#     results_dir='results'
#     max_time=300
#     sd_method='dual_simplex'
#     reset=False,
#     partition_polytope=False
#     n=0
#     k=0
#     spindle=False
#     spindle_dim=0
#     n_cone_facets=0
#     n_parallel_facets=0

#     ### Build Initial Polyhedron from file
#     print(f'Reading {mps_fn}...')
#     c, B, d, A, b = read_mps_preprocess(mps_fn)
#     print('Building polyhedron...')
#     P = Polyhedron(B, d, A, b, c)

In [3]:
problem_dir = 'netlib_lp_subset'
# problem = 'kb2'
# problem = 'maros'
# problem = 'pilot4'
# problem = 'adlittle'
# problem = 'angela_test_prob'
# problem = 'network_ex'
# problem = 'gfrd-pnc'
# problem = 'modszk1'
# problem = 'cycle'
# problem = 'cre-c'
# problem = 'qap08'
# problem = 'grow7'
problem = 'standmps'
mps_fn=os.path.join(problem_dir, problem)
results_dir='results'
max_time=300
sd_method='dual_simplex'
reset=False,
partition_polytope=False
n=0
k=0
spindle=False
spindle_dim=0
n_cone_facets=0
n_parallel_facets=0

## Perform SDCA

### Original Code

In [4]:
### Build Initial Polyhedron from file
print(f'Reading {mps_fn}...')
c, B, d, A, b = read_mps_preprocess(mps_fn)
print('Building polyhedron...')
P = Polyhedron(B, d, A, b, c)

print('Finding feasible solution...')
x_feasible, vbasis, cbasis = P.find_feasible_solution(verbose=False)

x = x_feasible
c=None
verbose=False
method='dual_simplex'
reset=False
max_time=300
first_warm_start=None
save_first_steps=0
problem_name=''

if c is not None:
    P.set_objective(c)

t0 = time.time()
x_current = x
if save_first_steps:
    np.save('solutions/{}_0.npy'.format(problem_name), x_current)      
active_inds = P.get_active_constraints(x_current)
# print(len(active_inds))
    
pm = P.build_polyhedral_model(active_inds=active_inds, method=method)

if first_warm_start is not None:
    print('Using custom warm start')
    pm.set_solution(first_warm_start)
t1 = time.time()
build_time = t1 - t0
print('Polyhedral model build time: {}'.format(build_time))
    
sub_times = {'sd': [], 'step': [], 'solve': [], 'phase_times': []}    
descent_circuits = []
obj_values = []
step_sizes = []
iter_times = []
simplex_iters = []
pts_visited = []
iteration = 0
obj_value = P.c.dot(x_current)
obj_values.append(obj_value)
t2 = time.time()
iter_times.append(t2 - t1)


# compute steepest-descent direction
descent_direction, y_pos, y_neg, steepness, num_steps, solve_time, phase_times = pm.compute_sd_direction(verbose=verbose)
simplex_iters.append(num_steps)
sub_times['solve'].append(solve_time)
sub_times['phase_times'].append(phase_times)
    
t3 = time.time()
sub_times['sd'].append(t3 - t2)
    
while abs(steepness) > EPS:
        
    t3 = time.time()
    if reset:
        pm.reset()
        
    # take maximal step
    pts_visited.append(x_current)
    x_current, alpha, active_inds = P.take_maximal_step(descent_direction, y_pos, y_neg) 
        
    if iteration % 50 == 0 or iteration == 1:
        print('\nIteration {}'.format(iteration))
        print('Objective: {}'.format(obj_value))
        print('Steepness: {}'.format(steepness))
        print('Step length: {}'.format(alpha))
        
    t4 = time.time()
    obj_value = P.c.dot(x_current)
    obj_values.append(obj_value)
    iter_times.append(t4 - t1)
    sub_times['step'].append(t4 - t3) 
    descent_circuits.append(descent_direction)
    step_sizes.append(alpha)     
                
    if math.isinf(alpha):
        # problem is unbounded
        result(status=1, circuits=descent_circuits, steps=step_sizes)
        
    pm.set_active_inds(active_inds)
    descent_direction, y_pos, y_neg, steepness, num_steps, solve_time, phase_times = pm.compute_sd_direction(
                                                                                                verbose=verbose)
        
    t5 = time.time()
    sub_times['sd'].append(t5 - t4)
    sub_times['solve'].append(solve_time)
    sub_times['phase_times'].append(phase_times)
    simplex_iters.append(num_steps)
        
    iteration += 1
    current_time = t5 - t1
    if current_time > max_time:
        result(status=2)
    if iteration <= save_first_steps:
        np.save('solutions/{}_{}.npy'.format(problem_name, iteration), x_current)

t6 = time.time()
total_time = t6 - t1   
print('Total time for steepest-descent scheme: {}'.format(total_time))

Reading netlib_lp_subset\standmps...
Building polyhedron...
Problem size: n = 1075,  m_B = 1362,  m_A = 284
Finding feasible solution...
Set parameter Username
Academic license - for non-commercial use only - expires 2025-04-01
Building polyhedral model. Solve method: dual_simplex
Set parameter Method to value 1
Polyhedral model built!
Polyhedral model build time: 0.5222792625427246

Iteration 0
Objective: 1469.9374999999998
Steepness: -0.5604395604395604
Step length: 0.38674999999999177

Iteration 1
Objective: 1469.7207499999997
Steepness: -0.5524475524475524
Step length: 14.478750000000002
Total time for steepest-descent scheme: 0.14034366607666016


In [5]:
# print(obj_values[-1])
# print(x_current)
for i in range(len(pts_visited)-1):
    print((pts_visited[i+1] == pts_visited[i]))

[ True False False ...  True  True  True]
[ True  True  True ...  True  True  True]
[ True  True  True ...  True  True  True]
[ True  True  True ...  True  True  True]
[ True  True  True ...  True  True  True]
[ True  True  True ...  True  True  True]
[ True  True  True ...  True  True  True]
[ True  True  True ...  True  True  True]
[ True  True  True ...  True  True  True]
[ True  True  True ...  True  True  True]
